In [54]:
from tensorflow.keras.models import load_model
model1 = load_model(r'C:\Users\ThinkPad\Downloads\CaptureCam\model_img.h5')


In [55]:
!curl -O https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
!curl -O https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 28104  100 28104    0     0    99k      0 --:--:-- --:--:-- --:--:--   99k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  7 10.1M    7  832k    0     0  1117k      0  0:00:09 --:--:--  0:00:09 1118k
 69 10.1M   69 7216k    0     0  4134k      0  0:00:02  0:00:01  0:00:01 4135k
100 10.1M  100 10.1M    0     0  4471k      0  0:00:02  0:00:02 --:--:-- 4476k


In [56]:
import cv2
print("[INFO] loading model...")
prototxt = 'deploy.prototxt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

[INFO] loading model...


In [58]:
import imutils
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    if ret:
        
        frame = imutils.resize(frame, width=400)
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 1)
                minX, maxX = min(startX, endX), max(startX, endX)
                minY, maxY = min(startY, endY), max(startY, endY)

                mask_label = {0:'MASK INCORRECT',1:'MASK', 2:'NO MASK'}
                color_label = {0:(0,255,0),1:(0,255,255), 2:(255,0,0)}
              
                resized_face = frame[minY:maxY, minX:maxX].copy()
                resized_face = cv2.cvtColor(resized_face,cv2.COLOR_RGB2BGR)
                resized_face = cv2.resize(resized_face, (35, 35))
                reshaped_face = np.reshape(resized_face,[1,35,35,3])/255.0
                
                face_result = model1.predict(reshaped_face)
                
                predict = np.argmax(face_result, axis=1)
                if predict == 0:
                    cv2.putText(frame,"MASK INCORRECT", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                if predict == 1:
                    cv2.putText(frame,"MASK", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)
                if predict == 2:
                    cv2.putText(frame,"NO MASK", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
        
        cv2.imshow("frame", frame)
        
        key = cv2.waitKey(1)

        if key == 27:

            break

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step
